In [38]:
import math 
import numpy as np
from astropy.time import Time, TimeDelta
from astropy import units as u
from numpy import random, mean, var, std
import scipy as sp
from scipy import signal, misc, linalg as lng
from numpy import array, arange, abs as np_abs, polynomial as P
from numpy.fft import rfft, rfftfreq
from numpy.random import uniform
from math import sin, pi
from scipy import signal
import os
import ephem
import glob
import progress
from progress.bar import IncrementalBar
from astropy.utils import iers
iers.conf.auto_download = False  
import json


In [3]:
def culmin(a,d):

    rg=1.7453292e-2
    rm=2.90888e-4
    rs=4.8481368e-6
    fi=0.956829
    be=0.008436
    alf=(a[0]*rg+a[1]*rm+a[2]*rs)*15 #в радианы 
    delt=d[0]*rg+d[1]*rm+d[2]*rs
    aa=((math.sin(fi))**2)*((math.cos(be))**2)+(math.cos(fi))**2
    bb=2*(math.sin(fi))*(math.cos(be))*math.sin(delt)
    cc=(math.sin(delt))**2-(math.cos(fi))**2
    x=(bb+math.sqrt(bb**2-(4*aa*cc)))/(aa*2) 
    y=x*(math.sin(be))/math.cos(delt)
    z=math.sqrt(1-(y**2))
    q=y/z
    culm=alf-math.atan(q)
    if culm < 0:
            culm = 2*3.1415926+culm
    h=culm/(rg*15)
    nh=int(h)
    h=nh
    hm=(culm-h*rg*15)/(rm*15)
    nhm=int(hm)
    hm=nhm
    s=(culm-(h*rg*15)-(hm*rm*15))/(rs*15)
    iss=int(s+0.5) 
    return culm, hm, h, s, iss

In [33]:
F = glob.glob('/bsa/high-ground/2016/05_may/*.pnthr')
FILE = []
F.sort()
k = 0
for file in F: 
    head = {}
    with open(file, 'rb') as f:
        for i in range(16):
            line = f.readline()
            a, *b = line.decode("utf-8").strip('\n').split()
            head [a] = b

        a = [3,23,26.618] #координаты источника 
        d = [39,44,52.9]

        b = str(head['date_begin'][0])
        bb = [b[0:2:],b[3:5:],b[6:10:]]
        bb = list(reversed(bb))

        year = bb[0]
        month = bb[1]
        if int(month[0]) == 0: 
            month = month[1::]
        day = bb[2]
        if int(day[0]) == 0: 
            day = day[1::]

        t0 = int(year)+int(month)/12+int(day)/365 

        a,d,t0 = prec(a,d,t0)
        culm, hm, h, s, iss = culmin(a,d) 


        a1 = [h,hm,s]
        a6 = [h,hm,round(s)]
        d1 = d

        t1 = str(head['time_begin'][2])
        t2 = str(head['time_end'][0])
        dt = TimeDelta(3599, format='sec')
        try:
            tt1 = [t1[0:2:],t1[3:5:],t1[6:8:]]
            times1 = (str(bb[0] + '-' + bb[1] + '-'+ bb[2] + 'T'+ tt1[0] + ':' + tt1[1] + ':'+ tt1[2])) #ужас
            t_begin = Time(times1, format='isot', scale='utc', location=('37.36d', '54.50d')) #Москва
            t_end = t_begin + dt
         
        except:
            tt1 = [t1[0:1:],t1[2:4:],t1[5:7:]]
            times1 = (str(bb[0] + '-' + bb[1] + '-'+ bb[2] + 'T'+ tt1[0] + ':' + tt1[1] + ':'+ tt1[2])) #ужас
            t_begin = Time(times1, format='isot', scale='utc', location=('37.36d', '54.50d')) #Москва
            t_end = t_begin + dt
           
                
        startime_begin = (t_begin.sidereal_time('apparent'))
        startime_end = (t_end.sidereal_time('apparent'))

        st_begin = []
        st_end = []
        
        startime_begin = str(startime_begin)
        startime_end = str(startime_end)

        st_begin.append((startime_begin))
        st_end.append((startime_end))
        
        h1 = startime_begin[0:startime_begin.index('h'):]
        m1 = startime_begin[startime_begin.index('h')+1:startime_begin.index('m'):]
        s1 = startime_begin[startime_begin.index('m')+1:startime_begin.index('.'):]
        a = [int(h1),int(m1),round(float(s1))] 

        culm, hm, h, s, iss = culmin(a,d) 

        a2 = [h,hm,s]
        a3 = [h,hm,round(s)]
        d2 = d

        h2 = startime_end[0:startime_end.index('h'):]
        m2 = startime_end[startime_end.index('h')+1:startime_end.index('m'):]
        s2 = startime_end[startime_end.index('m')+1:startime_end.index('.'):]
        a = [int(h2),int(m2),round(float(s2))] 

        a,d,t0 = prec(a,d,t0)
        culm, hm, h, s, iss = culmin(a,d)

        a4 = [h,hm,s]
        a5 = [h,hm,round(s)]
        m = k
        
        a3_d = a3[0] + a3[1]/60 + a3[2]/360 #начало
        a6_d = a6[0] + a6[1]/60 + a6[2]/360 #источник
        a5_d = a5[0] + a5[1]/60 + a5[2]/360 #конец
        if a6_d > a3_d and a6_d < a5_d:
            FILE.append(file)  

        
        
f = open('/home/helen/File_list.txt', 'w')
simplejson.dump(FILE, f)
f.close()
